In [29]:
import pandas as pd
import re

In [37]:
df = pd.read_csv('../dataset/updated_imdb_data.csv')

def process_budget(budget):
    if pd.isna(budget):
        return ('USD', None) 
    currency = re.findall(r'[^\d.]+', str(budget))
    if '$' in budget:
        currency_symbol = 'USD'
    elif currency:
        currency_symbol = currency[0].strip()
    else:
        currency_symbol = 'USD'
    cleaned_budget = re.sub(r'[^\d.]+', '', str(budget))
    return (currency_symbol, cleaned_budget)

df['currency'], df['budget'] = zip(*df['budget'].apply(process_budget))
df.to_csv('../dataset/updated_imdb_data.csv', index=False)
currency_counts = df['currency'].value_counts()
print("Count values in 'currency':")
print(currency_counts)

Count values in 'currency':
currency
USD    18298
EUR     1377
INR      298
GBP      213
CAD      130
NOK       98
AUD       68
FRF       58
BRL       58
DKK       56
DEM       48
CNY       47
SEK       45
TRL       39
RUR       35
PLN       34
EGP       32
HKD       31
HUF       31
JPY       28
KRW       27
CZK       25
MXN       21
NZD       16
FIM       15
THB       15
SGD       14
ZAR       12
ARS       10
TWD       10
MYR        9
ITL        7
UAH        7
ESP        7
PKR        6
CHF        5
IRR        4
DOP        4
ISK        4
LVL        4
IDR        3
PHP        3
NLG        3
BDT        2
EEK        2
HRK        1
COP        1
ILS        1
AMD        1
IEP        1
SKK        1
NGN        1
CLP        1
LTL        1
YUM        1
BND        1
ATS        1
AED        1
Name: count, dtype: int64


In [38]:
"""with open('../dataset/country_codes.txt', 'r') as file:
    data = file.read()

rows = data.split('\n')
country_currency = []
for row in rows:
    if row.strip():
        parts = row.split('\t')
        currency_code = parts[0].strip()
        country_name = parts[1].split(',')[0].strip()  
        country_currency.append([currency_code, country_name])

df = pd.DataFrame(country_currency, columns=['Currency Code', 'Country'])

df.to_csv('../dataset/country_currency_codes.csv', index=False)"""

"with open('../dataset/country_codes.txt', 'r') as file:\n    data = file.read()\n\nrows = data.split('\n')\ncountry_currency = []\nfor row in rows:\n    if row.strip():\n        parts = row.split('\t')\n        currency_code = parts[0].strip()\n        country_name = parts[1].split(',')[0].strip()  \n        country_currency.append([currency_code, country_name])\n\ndf = pd.DataFrame(country_currency, columns=['Currency Code', 'Country'])\n\ndf.to_csv('../dataset/country_currency_codes.csv', index=False)"

# CPI Index Converter

In [39]:
data = pd.read_csv('../dataset/PRICES_CPI.csv') 
filtered_data = data[data['Subject'] == 'CPI: 01-12 - All items']
final_data = filtered_data[filtered_data['Measure'] == 'Index']
final_data.to_csv('../dataset/CPI_filtered.csv', index=False)

data = pd.read_csv('CPI_filtered.csv')  

# Ottieni i valori unici della colonna 'frequency'
unique_frequencies = data['Frequency'].unique()

# Stampa i valori unici
print("Unique values of 'frequency':", unique_frequencies)



C:\Users\andre\AppData\Local\Temp\ipykernel_25028\3660450759.py:1: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../dataset/PRICES_CPI.csv')


Unique values of 'frequency': ['Quarterly' 'Monthly' 'Annual']


Create means for each year of each country

In [40]:
data['Means'] = pd.NA

# Compute measn
def calculate_annual_mean(group):
    if group['Frequency'].iloc[0] == 'Annual':
        group['Means'] = group['Value']
    elif group['Frequency'].iloc[0] == 'Quarterly':
        group['Means'] = group.groupby(group['TIME'].str[:4])['Value'].transform('mean')
        group = group[group['TIME'].str.endswith('Q1')]
    elif group['Frequency'].iloc[0] == 'Monthly':
        group['Means'] = group.groupby(group['TIME'].str[:4])['Value'].transform('mean')
        group = group[group['TIME'].str.endswith('01')]
    return group

result = data.groupby(['Country', 'Frequency']).apply(calculate_annual_mean)

result.to_csv('../dataset/CPI_filtered.csv', index=False)

C:\Users\andre\AppData\Local\Temp\ipykernel_25028\3734416054.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = data.groupby(['Country', 'Frequency']).apply(calculate_annual_mean)


code

In [56]:
new_imdb_currency = pd.read_csv('../dataset/updated_imdb_data.csv')
cpi_filtered = pd.read_csv('CPI_filtered.csv')
# , header=None, delimiter=';'
country_currency_code = pd.read_csv('../dataset/NEW_country_currency_codes.csv', delimiter=';')
country_currency_code.columns = ['currency_code', 'country']
currency_to_country = country_currency_code.set_index('currency_code')['country'].to_dict()
new_imdb_currency['nation'] = new_imdb_currency['currency'].map(currency_to_country)
#Format year column from decimal to integer
new_imdb_currency['year'] = new_imdb_currency['year'].astype(int)
new_imdb_currency.to_csv('../dataset/updated_imdb_data.csv', index=False)


Inflation conversion for USD movies

In [57]:
inflation_data = pd.read_csv('../dataset/inflation.csv')
#mean computing
inflation_data['Mean'] = inflation_data.iloc[:, 2:].mean(axis=1)  
inflation_data = inflation_data[['Year', 'Mean']]
#print(inflation_data.head())
inflation_data.to_csv('../dataset/adjusted_inflation.csv', index=False)

In [58]:
# inflation calculus on budget
movies = pd.read_csv('../dataset/updated_imdb_data.csv')
# cpi_data = pd.read_csv('CPI_filtered.csv')
cpi_data = pd.read_csv('../dataset/CPI_filtered.csv')
cpi_usa = pd.read_csv('../dataset/adjusted_inflation.csv')
cpi_usa['Year'] = cpi_usa['Year'].astype(int)

# Preparazione del DataFrame aggiungendo la nuova colonna
movies['budget_with_CPI'] = 0
failures = 0
usa_2023_cpi = 304.65 # cpi of USA in 2023

def get_cpi(data, country, year):
    cpi = data[(data['Country'] == country) & (data['TIME'].str.startswith(str(year)))]
    if not cpi.empty:
        return cpi.iloc[0]['Means']
    return None

def get_usa_cpi(cpi_data, film_year):
    #cpi = data[data['Year'].str.startswith(str(film_year))]
    cpi = cpi_data[cpi_data['Year'] == film_year]
    if not cpi.empty:
        return cpi.iloc[0]['Mean']  
    return None

for index, row in movies.iterrows():
    nation = row['nation']
    film_year = row['year']
    budget = row['budget']

    if nation == 'United States':
        cpi_film_year = get_usa_cpi(cpi_usa, film_year)
    
        if cpi_film_year is None:
            reason = ""
            if cpi_film_year is None: 
                reason = "CPI FILM YEAR,"
            else:
                reason = "CPI 2023,"
            print(f"{reason} Failed to find CPI data for {nation} in year {film_year} or 2023")
            failures += 1
            movies.drop(index, inplace=True)
            continue
    
        if cpi_film_year > 0: 
            adjusted_budget = (budget * usa_2023_cpi) / cpi_film_year
            movies.at[index, 'budget_with_CPI'] = adjusted_budget
    else:
        cpi_film_year = get_cpi(cpi_data, nation, film_year)
        cpi_2023 = get_cpi(cpi_data, nation, 2023)
    
        if cpi_film_year is None or cpi_2023 is None:
            reason = ""
            if cpi_film_year is None: 
                reason = "CPI FILM YEAR,"
            else:
                reason = "CPI 2023,"
            print(f"{reason} Failed to find CPI data for {nation} in year {film_year} or 2023")
            failures += 1
            movies.drop(index, inplace=True)
            continue
    
        if cpi_film_year > 0:  # Evita la divisione per zero
            adjusted_budget = (budget * cpi_2023) / cpi_film_year
            movies.at[index, 'budget_with_CPI'] = adjusted_budget

print(f"Total failures: {failures}")

movies.to_csv('../dataset/inflation_update_imdb_data.csv', index=False)

C:\Users\andre\AppData\Local\Temp\ipykernel_25028\3054974163.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '175700.33726812815' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  movies.at[index, 'budget_with_CPI'] = adjusted_budget


CPI FILM YEAR, Failed to find CPI data for Norway in year 1921 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1927 or 2023
CPI FILM YEAR, Failed to find CPI data for nan in year 1927 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1935 or 2023
CPI FILM YEAR, Failed to find CPI data for nan in year 1939 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1943 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1944 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1945 or 2023
CPI FILM YEAR, Failed to find CPI data for United Kingdom in year 1946 or 2023
CPI FILM YEAR, Failed to find CPI data for Sweden in year 1951 or 2023
CPI FILM YEAR, Failed to find CPI data for Sweden in year 1953 or 2023
CPI FILM YEAR, Failed to find CPI data for Japan in year 1954 or 2023
CPI FILM YEAR, Failed to find CPI data for nan in year 1956 or 2023
CPI FILM YEAR, Failed to find CPI data 

Now focus on revenue

In [59]:
data = pd.read_csv('../dataset/inflation_update_imdb_data.csv')

error_count = 0

for index, row in data.iterrows():
    try:
        income = row['worlwide_gross_income'].replace('$', '').replace(' ', '')
        data.at[index, 'worlwide_gross_income'] = float(income)
    except ValueError:
        error_count += 1
        data.drop(index, inplace=True)

# Stampa il numero di errori incontrati
print(f"Total conversion errors: {error_count}")

# Salva il DataFrame modificato
data.to_csv('../dataset/inflation_update_imdb_data.csv', index=False)

Total conversion errors: 5


In [ ]:
movies = pd.read_csv('../dataset/inflation_update_imdb_data.csv')
cpi_usa = pd.read_csv('../dataset/adjusted_inflation.csv')
cpi_usa['Year'] = cpi_usa['Year'].astype(int)

movies['revenue_with_CPI'] = 0
failures = 0
usa_2023_cpi = 304.65 # cpi of USA in 2023

for index, row in movies.iterrows():
    nation = row['nation']
    film_year = row['year']
    revenue = row['worlwide_gross_income']

    # Salta le righe dove la nazione è 'United States'
    cpi_film_year = get_usa_cpi(cpi_usa, film_year)
    
    if cpi_film_year is None:
        reason = ""
        if cpi_film_year is None: 
            reason = "CPI FILM YEAR,"
        else:
            reason = "CPI 2023,"
        print(f"{reason} Failed to find CPI data for {nation} in year {film_year} or 2023")
        failures += 1
        movies.drop(index, inplace=True)
        continue
    
    print(revenue)
    print(cpi_film_year)
    if cpi_film_year > 0:  # Evita la divisione per zero
        adjusted_budget = (revenue * usa_2023_cpi) / cpi_film_year
        movies.at[index, 'revenue_with_CPI'] = adjusted_budget

# Stampa il numero di righe che non hanno trovato dati CPI
print(f"Total failures: {failures}")

movies.to_csv('../dataset/inflation_update_imdb_data.csv', index=False)

Currency conversion

In [63]:
df_films = pd.read_csv('../dataset/inflation_update_imdb_data.csv')
df_exchange_rates = pd.read_csv('../dataset/eurofxref-hist.csv')

def add_default_date(date_str):
    if pd.notnull(date_str) and len(date_str) == 4:
        return f"{date_str}-01-01"
    return date_str

df_films['date_published'] = df_films['date_published'].apply(add_default_date)

df_films['date_published'] = pd.to_datetime(df_films['date_published'], format='%Y-%m-%d', errors='coerce')

"""# Funzione per controllare e formattare la data
def format_date(date):
    if len(date) == 4:  # Significa che abbiamo solo l'anno
        return pd.to_datetime(date + '-01-01', format='%Y-%m-%d')
    else:
        return pd.to_datetime(date, format='%Y-%m-%d', errors='coerce')
 
# Convertire le colonne date_published e date in formato datetime
df_films['date_published'] = pd.to_datetime(df_films['date_published'].astype(str) + '-01-01', format='%Y-%m-%d',errors='coerce')"""

df_exchange_rates['Date'] = pd.to_datetime(df_exchange_rates['Date'])

def find_closest_date(date, dates):
    return dates.iloc[(dates - date).abs().argsort()[0]]

converted_budgets = []
missing_currencies = set()
error_count = 0

for index, film in df_films.iterrows():
    closest_date = find_closest_date(film['date_published'], df_exchange_rates['Date'])
    exchange_rate_row = df_exchange_rates[df_exchange_rates['Date'] == closest_date]
 
    if film['currency'] != 'USD':
        if film['currency'] in exchange_rate_row.columns or film['currency'] == 'EUR':
            if film['currency'] != 'EUR':
                currency_rate = exchange_rate_row[film['currency']].values[0]
            else:
                currency_rate = 1
            eur_rate = exchange_rate_row['USD'].values[0]
            converted_budget = (film['budget_with_CPI'] / currency_rate) * eur_rate
        else:
            missing_currencies.add(film['currency'])
            error_count += 1
            converted_budget = None  
    else:
        converted_budget = film['budget_with_CPI']
 
    converted_budgets.append(converted_budget)
 
df_films['converted_budget'] = converted_budgets
 
df_films.to_csv('../dataset/updated_films.csv')

In [64]:
print(f"Total errors: {error_count}")
if missing_currencies:
    print("Missing currencies:", missing_currencies)

Total errors: 2
Missing currencies: {'CLP', 'COP'}


In [69]:
df = pd.read_csv('../dataset/updated_films.csv')

df = df.query("currency != 'COP' and currency != 'CLP'")

df.to_csv('../dataset/updated_films.csv', index=False)

df_sorted = df.sort_values(by='revenue_with_CPI', ascending=False)

print(df_sorted.head(50))

df['revenue_with_CPI'] = df['revenue_with_CPI'].round(1)
df['converted_budget'] = df['converted_budget'].round(1)

df.to_csv('../dataset/rounded_updated_films.csv', index=False)

       Unnamed: 0 imdb_title_id  \
182           182     tt0031381   
183           183     tt0031383   
8960         8960     tt0413348   
5630         5630     tt0145682   
7663         7663     tt0330505   
5857         5857     tt0167281   
319           319     tt0039000   
9788         9788     tt0459083   
235           235     tt0034493   
9637         9637     tt0451475   
6112         6112     tt0198811   
5162         5162     tt0120347   
10424       10424     tt0499556   
5161         5161     tt0120338   
10422       10422     tt0499549   
162           162     tt0029583   
163           163     tt0029584   
1324         1324     tt0076759   
1325         1325     tt0076760   
5471         5471     tt0129403   
18512       18514     tt4156152   
15759       15760     tt2017464   
11000       11000     tt0848477   
6685         6685     tt0259492   
382           382     tt0042333   
18509       18511     tt4154796   
5787         5787     tt0163315   
6194         6194   